In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.svm import OneClassSVM

In [4]:
import matplotlib.pyplot as plt

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
df = pd.read_csv('records_v2.csv')

In [11]:
print(df.columns.tolist())

['id', 'reading', 'reading_time', 'Fuel Volume (L)']


In [12]:
df['Fuel Volume (L)'] = pd.to_numeric(df['Fuel Volume (L)'], errors='coerce')

In [13]:
df['reading_time'] = pd.to_datetime(df['reading_time'], errors='coerce')

In [14]:
df = df.sort_values(['reading_time', 'id']).reset_index(drop=True)

In [15]:
df['volume_diff'] = df['Fuel Volume (L)'].diff()

In [16]:
df['prev_fuel_volume'] = df['Fuel Volume (L)'].shift(1)

In [17]:
df['rolling_mean'] = df['Fuel Volume (L)'].rolling(window=10, min_periods=5).mean()
df['rolling_std'] = df['Fuel Volume (L)'].rolling(window=10, min_periods=5).std()

In [18]:
print("✅ Data preparation complete")
print(f"Dataset shape: {df.shape}")
print(f"Features: volume_diff, prev_fuel_volume, rolling_mean, rolling_std")

✅ Data preparation complete
Dataset shape: (1228, 8)
Features: volume_diff, prev_fuel_volume, rolling_mean, rolling_std


In [19]:
feature_cols = ['volume_diff', 'rolling_mean', 'rolling_std']
X = df[feature_cols].dropna()

In [20]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
from sklearn.svm import OneClassSVM

# 1) create the model  (NO random_state here)
ocsvm = OneClassSVM(kernel='rbf', nu=0.03, gamma='scale')

# 2) fit the model on your scaled features
ocsvm.fit(X_scaled)          # <-- this line is missing in your screenshot

# 3) then predict and save flags
df.loc[X.index, 'svmflag'] = ocsvm.predict(X_scaled)